In [ ]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Movie Review Sentiment with BERT on TF Hub

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==1.15
!pip install bert-tensorflow==1.0.1

Found existing installation: tensorflow 1.15.0
Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
    /usr/local/lib/python3.7/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0
  Using cached tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl (412.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.15.0 which is incompatible.


In [ ]:
from google.colab import drive
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
# import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras
import os
import re
import pandas as pd
import numpy as np

In [ ]:
import bert

from bert import run_classifier
from bert import optimization
from bert import tokenization

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow import keras
import os
import re
import pandas as pd
import numpy as np

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:

import os
print(os.listdir())
os.chdir(r'./gdrive/MyDrive/congress_SSL')


['.config', 'gdrive', 'sample_data']


In [ ]:
print(os.listdir())

['somas', 'Bert_somas.ipynb', 'congress_data', 'results', 'labelspreading.ipynb', 'saved_bert_model', 'Bert prediction.ipynb', 'saved_bert_model2', 'Bert vectors.ipynb']


Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [ ]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = r'./saved_bert_model2/'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: ./saved_bert_model2/ *****


In [ ]:
print(os.listdir())

['somas', 'Bert_somas.ipynb', 'congress_data', 'results', 'labelspreading.ipynb', 'saved_bert_model', 'Bert prediction.ipynb', 'saved_bert_model2', 'Bert vectors.ipynb']


#Data

First, let's download the dataset, hosted by Stanford. The code below, which downloads, extracts, and imports the IMDB Large Movie Review Dataset, is borrowed from [this Tensorflow tutorial](https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub).

In [ ]:


# # Load all files from a directory in a DataFrame.
# def load_directory_data(directory):
#   data = {}
#   data["sentence"] = []
#   data["sentiment"] = []
#   for file_path in os.listdir(directory):
#     with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
#       data["sentence"].append(f.read())
#       data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
#   return pd.DataFrame.from_dict(data)

# # Merge positive and negative examples, add a polarity column and shuffle.
# def load_dataset(directory):
#   pos_df = load_directory_data(os.path.join(directory, "pos"))
#   neg_df = load_directory_data(os.path.join(directory, "neg"))
#   pos_df["polarity"] = 1
#   neg_df["polarity"] = 0
#   return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# # Download and process the dataset files.
# def download_and_load_datasets(force_download=False):
#   dataset = tf.keras.utils.get_file(
#       fname="aclImdb.tar.gz", 
#       origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
#       extract=True)
  
#   train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
#                                        "aclImdb", "train"))
#   test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
#                                       "aclImdb", "test"))
  
#   return train_df, test_df


In [ ]:
congress = pd.read_csv(r'./congress_data/crs_2009_20_10_04_trancated.csv',  sep ='\t', encoding = 'ISO-8859-1', index_col = 0)

In [ ]:
dws = np.load(r'./congress_data/dws_2009_20.npy')

In [ ]:
congress['dw']= dws

In [ ]:
congress.head()

,speech_id,true_id,chamber,state,party,congress,daysafter,monthsafter,speech,date,topics_prior,dw
594060,1110000007,L000557,H,CT,D,111,6578,216,our democracy renews itself every 2 years as m...,20090106,22,-0.398
594061,1110000009,P000587,H,IN,R,111,6578,216,madam clerk. as chairman of the republican con...,20090106,22,0.655
594062,1110000013,B000589,H,OH,R,111,6578,216,. leader hoyer. fellow members. and a special ...,20090106,18,0.513
594063,1110000014,P000197,H,CA,D,111,6578,216,thank you very much. leader boehner. together....,20090106,22,-0.490
594064,1110000016,L000557,H,CT,D,111,6578,216,"for""your nomination this morning. thank you to...",20090106,-1,-0.398


In [ ]:

true_switchers_ = ['A000361','B000229', 'B001264', 'D000168', 'G000280', 'H000067', 'H000390', 'L000119', 'P000066', 'T000058', 'C000077','F000257', 'G000557', 
'S000320', 'S000709','J000072']

independents = ['S000033', 'B001237', 'K000383']

true_switchers =independents + true_switchers_

In [ ]:
# filter out all switchers
congress_flt_ = congress.loc[~congress['true_id'].isin(true_switchers)]
congress_flt_['is_Dem'] = (congress_flt_['party'] == 'D')*1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# for test, comment this for real training


# congress_flt = congress_flt_.sample(5000)




In [ ]:
congress_flt = congress_flt_.sort_values(by = ['dw'], ascending = True, ignore_index = True)

In [ ]:
congress_flt.to_csv(r'./congress_data/congress_sorted_2009_20_10_04.csv', sep = '\t', index_label= False, encoding = 'ISO-8859-1')

In [ ]:
congress_flt[:5], congress_flt[-5:]

(    speech_id  true_id chamber state  ...      date  topics_prior     dw  is_Dem
 0  1130067801  W000817       S    MA  ...  20140107            23 -0.769       1
 1  1130082890  W000817       S    MA  ...  20140327            45 -0.769       1
 2  1140095053  W000817       S    MA  ...  20160523            64 -0.769       1
 3  1130074735  W000817       S    MA  ...  20140212            18 -0.769       1
 4  1130074743  W000817       S    MA  ...  20140212            64 -0.769       1
 
 [5 rows x 13 columns],
          speech_id  true_id chamber  ... topics_prior     dw  is_Dem
 206894  1130048513  L000577       S  ...           28  0.916       0
 206895  1140093911  L000577       S  ...           30  0.916       0
 206896  1140004428  L000577       S  ...           -1  0.916       0
 206897  1140094358  L000577       S  ...           24  0.916       0
 206898  1140070318  L000577       S  ...           22  0.916       0
 
 [5 rows x 13 columns])

In [ ]:
def sample(df, biased = True, notmask = 0.8):
  parties = df['is_Dem'].values
  topics = df['topics_prior'].values
  total = len(df)
  print(total)
  if not biased: 
    print('started random split, test size {}'.format(1-notmask))
    sss = StratifiedShuffleSplit(n_splits=5, test_size= 1- notmask)
    trainid, testid = next(sss.split(topics, parties))
    assert len(trainid) + len(testid) == total, "length error"
  else:
    print('started biased split, test size {}'.format(1-notmask))
    topn = int(total * notmask/2.0)
    testid = np.arange(total)[topn:-topn]
    trainid = np.concatenate([np.arange(total)[:topn], np.arange(total)[-topn:]])
  return trainid, testid

In [ ]:
trainid, testid = sample(congress_flt, biased = False, notmask = 0.1)

206899
started random split, test size 0.9


In [ ]:


# train, test = download_and_load_datasets()

In [ ]:
congress_flt.columns

Index(['speech_id', 'true_id', 'chamber', 'state', 'party', 'congress',
       'daysafter', 'monthsafter', 'speech', 'date', 'topics_prior', 'dw',
       'is_Dem'],
      dtype='object')

For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [ ]:
DATA_COLUMN = 'speech'
LABEL_COLUMN = 'is_Dem'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [ ]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
# train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
#                                                                    text_a = x[DATA_COLUMN], 
#                                                                    text_b = None, 
#                                                                    label = x[LABEL_COLUMN]), axis = 1)

# test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
#                                                                    text_a = x[DATA_COLUMN], 
#                                                                    text_b = None, 
#                                                                    label = x[LABEL_COLUMN]), axis = 1)
traintest_InputExamples = congress_flt.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [ ]:
traintest_InputExamples[:5]

0    <bert.run_classifier.InputExample object at 0x...
1    <bert.run_classifier.InputExample object at 0x...
2    <bert.run_classifier.InputExample object at 0x...
3    <bert.run_classifier.InputExample object at 0x...
4    <bert.run_classifier.InputExample object at 0x...
dtype: object

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [ ]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Great--we just learned that the BERT model we're using expects lowercase data (that's what stored in tokenization_info["do_lower_case"]) and we also loaded BERT's vocab file. We also created a tokenizer, which breaks words into word pieces:

In [ ]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer. And I love this.\n But I think is tsrerereazrsaf")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer',
 '.',
 'and',
 'i',
 'love',
 'this',
 '.',
 'but',
 'i',
 'think',
 'is',
 'ts',
 '##rer',
 '##ere',
 '##az',
 '##rsa',
 '##f']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [ ]:
# We'll set sequences to be at most 256 tokens long.
MAX_SEQ_LENGTH = 150
# Convert our train and test features to InputFeatures that BERT understands.
traintest_features = np.array(bert.run_classifier.convert_examples_to_features(traintest_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer))
# train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

# test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 206899


INFO:tensorflow:Writing example 0 of 206899


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


KeyboardInterrupt: ignored

#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [ ]:
def create_model( input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=False)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer_ = bert_outputs["pooled_output"]
  output_layer = tf.identity(output_layer_, name = 'output')
  out = tf.get_default_graph().get_tensor_by_name('output:0')
  return out

  # hidden_size = output_layer.shape[-1].value

  # # Create our own layer to tune for politeness data.
  # output_weights = tf.get_variable(
  #     "output_weights", [num_labels, hidden_size],
  #     initializer=tf.truncated_normal_initializer(stddev=0.02))

  # output_bias = tf.get_variable(
  #     "output_bias", [num_labels], initializer=tf.zeros_initializer())

  # with tf.variable_scope("loss"):

  #   # Dropout helps prevent overfitting
  #   output_layer_ = tf.nn.dropout(output_layer, keep_prob=0.9)

  #   logits = tf.matmul(output_layer_, output_weights, transpose_b=True)
  #   logits = tf.nn.bias_add(logits, output_bias)
  #   log_probs = tf.nn.log_softmax(logits, axis=-1)

  #   # Convert labels into one-hot encoding
  #   one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

  #   predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
  #   # If we're predicting, we want predicted labels and the probabiltiies.
  #   if is_predicting:
  #     return  output_layer
  #   # If we're train/eval, compute loss between predicted and actual label
  #   per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
  #   loss = tf.reduce_mean(per_example_loss)
  #   return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [ ]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    # is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # # TRAIN and EVAL
    # if not is_predicting:

    #   (loss, predicted_labels, log_probs) = create_model(
    #     is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

    #   train_op = bert.optimization.create_optimizer(
    #       loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

    #   # Calculate evaluation metrics. 
    #   def metric_fn(label_ids, predicted_labels):
    #     accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
    #     f1_score = tf.contrib.metrics.f1_score(
    #         label_ids,
    #         predicted_labels)
    #     auc = tf.metrics.auc(
    #         label_ids,
    #         predicted_labels)
    #     recall = tf.metrics.recall(
    #         label_ids,
    #         predicted_labels)
    #     precision = tf.metrics.precision(
    #         label_ids,
    #         predicted_labels) 
    #     true_pos = tf.metrics.true_positives(
    #         label_ids,
    #         predicted_labels)
    #     true_neg = tf.metrics.true_negatives(
    #         label_ids,
    #         predicted_labels)   
    #     false_pos = tf.metrics.false_positives(
    #         label_ids,
    #         predicted_labels)  
    #     false_neg = tf.metrics.false_negatives(
    #         label_ids,
    #         predicted_labels)
    #     return {
    #         "eval_accuracy": accuracy,
    #         "f1_score": f1_score,
    #         "auc": auc,
    #         "precision": precision,
    #         "recall": recall,
    #         "true_positives": true_pos,
    #         "true_negatives": true_neg,
    #         "false_positives": false_pos,
    #         "false_negatives": false_neg
    #     }

    #   eval_metrics = metric_fn(label_ids, predicted_labels)

    #   if mode == tf.estimator.ModeKeys.TRAIN:
    #     return tf.estimator.EstimatorSpec(mode=mode,
    #       loss=loss,
    #       train_op=train_op)
    #   else:
    #       return tf.estimator.EstimatorSpec(mode=mode,
    #         loss=loss,
    #         eval_metric_ops=eval_metrics)
    # else:
    out_layer = create_model( input_ids, input_mask, segment_ids, label_ids, num_labels)
    
    predictions = {
              'outlayer': out_layer
    }
    return tf.estimator.EstimatorSpec(mode, predictions=predictions)
  # Return the actual model function in the closure
  return model_fn


In [ ]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE =32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 5.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

In [ ]:
binwid = 8000
totalbins = int(len(traintest_features)/binwid)+1

for binno in range (totalbins+1):
  print(binno)
  low, high = [binno*binwid, binno*binwid +binwid]
  train_features = traintest_features[low:high]
  print(low, high)
  num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
  num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
  model_fn = model_fn_builder(  num_labels=len(label_list),
   learning_rate=LEARNING_RATE,
   num_train_steps=num_train_steps,
   num_warmup_steps=num_warmup_steps)

  estimator = tf.estimator.Estimator( model_fn=model_fn,
    # config=run_config,
    params={"batch_size": BATCH_SIZE})
  predict_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)
  # train_input_fn = 
  predictions = estimator.predict(predict_input_fn)
  all_berts = []
  for pre in predictions:
    i = pre['outlayer']
    # print(i.shape)
    # break
    all_berts.append(i)
  all_berts = np.array(all_berts)
  print(all_berts.shape)
  np.save(r'./results/all_berts_vec_{}'.format(binno), all_berts)
      

In [ ]:
vecs = []
for binno in range(26):
  vec = np.load(r'./results/all_berts_vec_{}.npy'.format(binno))
  print(vec.shape)
  if len(vec)>0:
    vecs.append(vec)

In [ ]:
all_vec = np.concatenate(vecs)

In [ ]:
all_vec.shape
np.save(r'./results/all_berts_vec', all_vec)

In [ ]:
# # Compute # train and warmup steps from batch size
# topn = 20000
# train_features = traintest_features[:topn]
# num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
# num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [ ]:
# # Specify outpit directory and number of checkpoint steps to save
# run_config = tf.estimator.RunConfig(
#     model_dir=OUTPUT_DIR,
#     save_summary_steps=SAVE_SUMMARY_STEPS,
#     save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
# model_fn = model_fn_builder(
#   num_labels=len(label_list),
#   learning_rate=LEARNING_RATE,
#   num_train_steps=num_train_steps,
#   num_warmup_steps=num_warmup_steps)

# estimator = tf.estimator.Estimator( model_fn=model_fn,
#   # config=run_config,
#   params={"batch_size": BATCH_SIZE})


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [ ]:
# Create an input function for prediction. drop_remainder = True for using TPUs.
predict_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)
# train_input_fn = 
predictions = estimator.predict(predict_input_fn)

In [ ]:
all_berts = []
for pre in predictions:
  i = pre['outlayer']
  # print(i.shape)
  # break
  all_berts.append(i)
all_berts = np.array(all_berts)
np.save(r'./results/all_berts_vec', all_berts)

In [ ]:
all_berts.shape

Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

In [ ]:
# print(f'Beginning Training!')
# current_time = datetime.now()
# estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
# print("Training took time ", datetime.now() - current_time)

Now let's use our test data to see how well our model did:

In [ ]:
# test_input_fn = run_classifier.input_fn_builder(
#     features=test_features,
#     seq_length=MAX_SEQ_LENGTH,
#     is_training=False,
#     drop_remainder=False)

In [ ]:
# a = estimator.evaluate(input_fn=test_input_fn, steps=None)

Now let's write code to make predictions on new sentences:

In [ ]:
# def getPrediction(in_sentences):
#   labels = ["Negative", "Positive"]
#   input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
#   input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
#   predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
#   predictions = estimator.predict(predict_input_fn)
#   return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [ ]:
# pred_sentences = [
#   "That movie was absolutely awful",
#   "The acting was a bit lacking",
#   "The film was creative and surprising",
#   "Absolutely fantastic!"
# ]

In [ ]:
# predictions = getPrediction(pred_sentences)

In [ ]:
# train on 90% non masked samples


model_dir = OUTPUT_DIR +  + r'getbert/'
tf.gfile.MakeDirs(model_dir)
run_config = tf.estimator.RunConfig(  model_dir=model_dir, save_summary_steps=SAVE_SUMMARY_STEPS,save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)
trainid, testid = sample(congress_flt, biased = False, notmask = 0.9)
train_features = traintest_features[trainid]
test_features = traintest_features[testid]


In [ ]:
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
# continue
model_fn = model_fn_builder(  num_labels=len(label_list),  learning_rate=LEARNING_RATE,  num_train_steps=num_train_steps, num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator( model_fn=model_fn, config=run_config,  params={"batch_size": BATCH_SIZE})
train_input_fn = bert.run_classifier.input_fn_builder( features=train_features,  seq_length=MAX_SEQ_LENGTH, is_training=True,   drop_remainder=False)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
test_input_fn = run_classifier.input_fn_builder(features=test_features,  seq_length=MAX_SEQ_LENGTH,    is_training=False,    drop_remainder=False)
testresult = estimator.evaluate(input_fn=test_input_fn, steps=None)
acc = testresult['eval_accuracy']

Voila! We have a sentiment classifier!

In [ ]:
# start predict

predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)


predictions = np.array(list(estimator.predict(predict_gen)))

In [ ]:
np.save(r'./congress_data/bert_vec_2009_20', predictions)